<a href="https://colab.research.google.com/github/satanimeet/SpamDetection/blob/main/cleandatabaseemotional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression



In [ ]:

df = pd.read_csv(database, encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
unnamed_cols = [col for col in df.columns if col.startswith('Unnamed:')]

# If there are unnamed columns, drop them
if unnamed_cols:
    df = df.drop(columns=unnamed_cols)
    print(f"Dropped the following unnamed columns: {unnamed_cols}")
else:
    print("No unnamed columns found to drop.")

# Display the head of the DataFrame again to confirm the columns are removed
df.head()

Dropped the following unnamed columns: ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
action_mapping = {'ham':0,'spam':1}
df['actions'] = df['v1'].map(action_mapping)
df = df.drop(['v1'], axis=1)
df.head()

,v2,actions
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
import re

def clean_text(text):
    """
    Cleans text by removing special characters, punctuation, and specific patterns.

    Args:
        text (str): The input text string.

    Returns:
        str: The cleaned text string.
    """
    if not isinstance(text, str):
        # Handle potential non-string values like NaN
        return text

    # Remove URLs (basic pattern)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove common symbols and punctuation (keeps letters, numbers, and spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove specific patterns like '###' or '$$$'
    # You can add more specific patterns here
    text = re.sub(r'#+|\$+', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Assuming your text content is in a column named 'v2'
# Apply the cleaning function to the text column
# Make sure to replace 'v2' with the actual name of your text column
df['cleaned_text'] = df['v2'].apply(clean_text)

# Display the head of the DataFrame to see the new cleaned column
df.head()

,v2,actions,cleaned_text
0,"Go until jurong point, crazy.. Available only ...",0,Go until jurong point crazy Available only in ...
1,Ok lar... Joking wif u oni...,0,Ok lar Joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...,0,U dun say so early hor U c already then say
4,"Nah I don't think he goes to usf, he lives aro...",0,Nah I dont think he goes to usf he lives aroun...


In [ ]:
!pip install transformers

In [ ]:
df.head()

,actions,cleaned_text
0,0,Go until jurong point crazy Available only in ...
1,0,Ok lar Joking wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor U c already then say
4,0,Nah I dont think he goes to usf he lives aroun...


In [ ]:
from transformers import AutoModel, AutoTokenizer

bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
df.to_csv('cleaned_emotionaldatabase.csv', index=False)